# 基礎班時問 1330

問1. 怎麼用python抓share point的資料

程式相關問題通常把問題翻譯成英文的簡短句子丟到 google 裡, 第一筆 stack overflow 的資料有 5, 6 成是解答

https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site

# 影像班時問 1430

In [ ]:
問1.  怎麼使用 GPU 加速 YOLO


- 1. 查 torch 版本
- 2. 安裝相對應的 CUDA toolkit
- 3. 重新安裝 torch 與 cuda 相關的套件

https://developer.nvidia.com/cuda-toolkit-archive/
https://pytorch.org/get-started/previous-versions/

cuda 12.1
pytorch 2.2.2
會使用以下設定
conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia


In [3]:
import torch

In [22]:
# 確認 cuda 可以用
torch.cuda.is_available()

True

In [2]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.pt')

In [21]:
model = model.to('cuda') # 這裡可以指定 使用 cuda

## yolo 模型使用

In [23]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model = model.to('cuda') # 這裡可以指定 使用 cuda


In [25]:
results = model("kaohsiung-traffic.jpg")


image 1/1 C:\Users\Charlie Lee\workspace\2024\2024-05-29\kaohsiung-traffic.jpg: 480x640 2 persons, 9 cars, 1 motorcycle, 1 bus, 2 trucks, 94.5ms
Speed: 3.0ms preprocess, 94.5ms inference, 711.5ms postprocess per image at shape (1, 3, 480, 640)


In [26]:
results[0].show()

## yolo 模型訓練步驟
- 準備資料
- 選擇模型
- 訓練
- 使用模型


In [27]:
# 選擇模型
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [28]:
import os
from bs4 import BeautifulSoup

def convert_voc_to_yolo(voc_dir, yolo_dir, classes):
    # 創建 YOLO 目錄（如果不存在）
    if not os.path.exists(yolo_dir):
        os.makedirs(yolo_dir)

    for filename in os.listdir(voc_dir):
        if not filename.endswith('.xml'):
            continue

        # 解析 XML 文件
        xml_path = os.path.join(voc_dir, filename)
        with open(xml_path) as f:
            data = f.read()
        soup = BeautifulSoup(data)

        # 獲取圖片尺寸
        width = int(soup.find('size').find('width').text)
        height = int(soup.find('size').find('height').text)

        # 打開 YOLO 格式的標註文件
        yolo_path = os.path.join(yolo_dir, filename.replace('.xml', '.txt'))
        with open(yolo_path, 'w') as f:
            for obj in soup.find_all('object'):
                # 獲取物件類別
                class_name = obj.find('name').text
                if class_name not in classes:
                    continue
                class_id = classes.index(class_name)

                # 獲取邊界框座標
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                # 轉換為 YOLO 格式
                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                box_width = (xmax - xmin) / width
                box_height = (ymax - ymin) / height

                # 寫入 YOLO 格式
                f.write(f'{class_id} {x_center} {y_center} {box_width} {box_height}\n')

In [30]:
# 定義類別列表（根據你的 Pascal VOC 資料集）
classes = ["with_mask", "without_mask", "mask_weared_incorrect"]  # 替換為你的類別名稱

# 設定 Pascal VOC 和 YOLO 目錄
voc_directory = 'datasets/annotations'
yolo_directory = 'datasets/yolo'

# 執行轉換
convert_voc_to_yolo(voc_directory, yolo_directory, classes)

In [ ]:
model.train(data='mask.yaml', epochs=5)

In [47]:
trained_model = YOLO('runs/detect/train42/weights/best.pt')

In [ ]:
results = trained_model('Masks_faces_courtesy-of-Ashley-Ruba.jpg')

In [60]:
results[0].show()

# 綜合時間 1530

In [62]:
import pandas as pd


In [83]:
last_name = 'Todd'
all_customers = pd.read_csv('customers.csv')[['customer_id', 'first_name', 'last_name']]
customers = all_customers[all_customers['last_name'] == last_name]

## https://raw.githubusercontent.com/pochangl/2024/main/2024-05-08/orders.csv
orders = pd.read_csv('orders.csv')[['order_id', 'customer_id', 'order_date']]

filtered_orders = pd.merge(left=customers, right=orders, on='customer_id')


In [84]:
groups = filtered_orders.groupby('customer_id').size()

In [86]:
type(groups)

pandas.core.series.Series

In [87]:
groups = groups.reset_index()
type(groups)

pandas.core.frame.DataFrame

In [89]:
groups.columns = ['customer_id', 'count']

In [90]:
groups

,customer_id,count
0,2,3
1,440,1


In [93]:
order_count = pd.merge(left=groups, right=all_customers, on='customer_id')
order_count = order_count[['last_name','first_name', 'count']]
order_count['name'] = order_count['first_name'] + ' ' + order_count['last_name']

In [92]:
order_count

,last_name,first_name,count,name
0,Todd,Kasha,3,Kasha Todd
1,Todd,Yevette,1,Yevette Todd
